In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
from datetime import timedelta 
import metpy.calc as mpcalc
import cftime
import warnings
import climlab.utils.thermo as climlab
import glob

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

df_list = []

# Full list of models and their shorter forms
modelfilename_list = ['CCSM4','CanESM2','CanESM5','E3SM-1-0','GFDL-CM4','HadGEM2','HadGEM3-GC31-LL','IPSL-CM6A-LR','MIROC-ES2L','MIROC-ESM',
                     'MIROC5','MIROC6','MPI-ESM','MRI-CGCM3','MRI-ESM2-0','UKESM1-0-LL']

modelname_list=['ccsm4','canam4','canesm5','e3sm','gfdl','hg2','hg3','ipsl','mies2l','miesm','mi5','mi6','mpi','mrcgcm','mresm','ukesm']
modelvar_list = ['ta','ts','hur','sfcWind','wap','uas','vas']
# Hardcoded cmip6
cmip6modelfilename_list = ['CanESM5','E3SM-1-0','GFDL-CM4','HadGEM3-GC31-LL','IPSL-CM6A-LR','MIROC-ES2L','MIROC6','MRI-ESM2-0','UKESM1-0-LL']
member_id = ['r1i1p1f1','r1i1p1f2','r1i1p1f3','r1i1p1f4']

cmip5modelfilename_list = []

# Make cmip5 list
for modelname in modelfilename_list:
    if modelname not in cmip6modelfilename_list:
        cmip5modelfilename_list.append(modelname)

# Define paths
root_path='/data/keeling/a/rytam2/a/4xCO2/'
kernel_path='/data/keeling/a/rytam2/a/kernels/gcms/*CCSM4*.nc'

# Files to select 
cmip6 = df[(df['source_id'].isin(cmip6modelfilename_list)) & (df['member_id'].isin(member_id)) & (df['variable_id'].isin(modelvar_list)) & \
   (df['experiment_id'] == 'abrupt-4xCO2') & (df['table_id'] == 'Amon') ]

# E3SM doesn't have uas/vas
e3sm_var = ['ua','va']
e3sm_uava = df[(df['source_id']=='E3SM-1-0') & (df['member_id'].isin(member_id)) & (df['variable_id'].isin(e3sm_var)) & \
   (df['experiment_id'] == 'abrupt-4xCO2') & (df['table_id'] == 'Amon') ]

# List of filepaths to get from cloud
zstore = np.append(cmip6.zstore.values,e3sm_uava.zstore.values)

# this only needs to be created once
gcs = gcsfs.GCSFileSystem(token='anon')

In [43]:
dict_ds_t700 = {}
dict_ds_ts = {}
dict_ds_hur700 = {}
dict_ds_sfcWind = {}
dict_ds_uas = {}
dict_ds_vas = {}
dict_ds_wap = {}
dict_ds_tadv = {}

for path in zstore:
        # Open each file 
        ds = xr.open_zarr(path, consolidated=True)
        
        modelname = ds.attrs['parent_source_id']
        print(modelname)

        # Select variable from dataset 
        if 'ta' in path:
            ds = ds.ta.sel(plev=70000) #select 700hPa 
        elif 'ts' in path:
            ds = ds.ts
        elif 'hur' in path:
            ds = ds.hur.sel(plev=70000)
        elif 'sfcWind' in path:
            ds = ds.sfcWind
        elif 'ua' in path:
            if 'E3SM' in modelname:
                ds = ds.ua.sel(plev=1e5)
            else: 
                ds = ds.uas
        elif 'va' in path:
            if 'E3SM' in modelname:
                ds = ds.va.sel(plev=1e5)
            else: 
                ds = ds.vas
        elif 'wap' in path: 
            ds = ds.wap.sel(plev=70000)
        

        # Change lon coords
        ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
        ds.lat.attrs['units'] = 'degrees' 
        ds.lon.attrs['units'] = 'degrees' 

        # Unify time coords to match the limited data in HadGEM2 (185912 - 200511 -- 146 years)
        ds = ds.isel(time=slice(0,146*12))
        
        #interp grids (selected CCSM4 for coordinates)
        kernel = xr.open_mfdataset(kernel_path)
        kernel_ds = kernel.dRdxi.isel(i=0).expand_dims({'time':ds.time}, axis=2)\
                    .assign_coords({'latitude':kernel.lat,'longitude':kernel.lon})\
                    .rename({'latitude':'lat','longitude':'lon'})
            
        # Interp variables and assign to dict 
        if 'ta' in path:
            # interp ts
            ds = ds.interp_like(kernel_ds)
            dict_ds_t700[modelname] = ds
        elif 'ts' in path:
            ds = ds.interp_like(kernel_ds)
            dict_ds_ts[modelname] = ds
        elif 'hur' in path:
            ds = ds.interp_like(kernel_ds)
            dict_ds_hur700[modelname] = ds
        elif 'sfcWind' in path:
            ds = ds.interp_like(kernel_ds)
            dict_ds_sfcWind[modelname] = ds
        elif 'ua' in path:
            ds = ds.interp_like(kernel_ds)
            dict_ds_uas[modelname] = ds
        elif 'va' in path:
            ds = ds.interp_like(kernel_ds)
            dict_ds_vas[modelname] = ds
        elif 'wap' in path:
            ds = ds.interp_like(kernel_ds)
            dict_ds_wap[modelname] = ds
            
            
        """if 'E3SM' in modelname or 'GFDL' in modelname:
            ds['time']=(ds.indexes['time']+timedelta(days=365*1849)).to_datetimeindex()#.normalize
        elif 'HadGEM3' in modelname or 'UKESM' in modelname: 
            ds['time']= ds.indexes['time'].to_datetimeindex()#.normalize
        elif 'MIROC-ES2L' in modelname or 'MRI-ESM2' in modelname or 'IPSL' in modelname: 
            ds['time']= ds.indexes['time']#.normalize
        elif 'MIROC6' in modelname:
            ds['time']=(ds.indexes['time']-timedelta(days=365*(1350+11/12))).to_datetimeindex()
        elif 'CanESM5' in modelname: 
            ds['time']=(ds.indexes['time']).to_datetimeindex()#.normalize
            """
        
        time_ds=xr.open_zarr(zstore[-12], consolidated=True)
        index=(time_ds.indexes['time'])[0:146*12]
        ds['time']=index

GFDL-CM4
GFDL-CM4
GFDL-CM4
GFDL-CM4
GFDL-CM4
GFDL-CM4
GFDL-CM4
IPSL-CM6A-LR
IPSL-CM6A-LR
IPSL-CM6A-LR
IPSL-CM6A-LR
IPSL-CM6A-LR
IPSL-CM6A-LR
IPSL-CM6A-LR
MRI-ESM2-0
MRI-ESM2-0
MRI-ESM2-0
MRI-ESM2-0
MRI-ESM2-0
MRI-ESM2-0
MRI-ESM2-0
UKESM1-0-LL
UKESM1-0-LL
UKESM1-0-LL
UKESM1-0-LL
UKESM1-0-LL
UKESM1-0-LL
UKESM1-0-LL
CanESM5
CanESM5
CanESM5
CanESM5
CanESM5
CanESM5
CanESM5
HadGEM3-GC31-LL
HadGEM3-GC31-LL
HadGEM3-GC31-LL
HadGEM3-GC31-LL
HadGEM3-GC31-LL
HadGEM3-GC31-LL
HadGEM3-GC31-LL
MIROC6
MIROC6
MIROC6
MIROC6
MIROC6
MIROC6
MIROC6
E3SM-1-0
E3SM-1-0
MIROC-ES2L
MIROC-ES2L
MIROC-ES2L
MIROC-ES2L
MIROC-ES2L
MIROC-ES2L
MIROC-ES2L
E3SM-1-0
E3SM-1-0
E3SM-1-0
E3SM-1-0
E3SM-1-0


In [42]:
modelvar_list = ['ta','ts','hur','sfcWind','wap','ua','va']
for modelname in cmip5modelfilename_list:

    for var in modelvar_list:
        # Get all files 
        filepath = glob.glob(root_path+'%s*%s*.nc'%(var,modelname))
        print(modelname,len(filepath),'\n',filepath,'\n')
        
        # If-loop Combine all .nc files as one dataset 
        if len(filepath)==1:
            ds = xr.open_mfdataset(filepath);
        elif len(filepath) > 1: 
            ds = xr.open_mfdataset(filepath,combine="by_coords");

        # Select variable from dataset 
        if var == 'ta':
            ds = ds.ta.sel(plev=70000) #select 700hPa 
        elif var == 'ts':
            ds = ds.ts
        elif var == 'hur':
            ds = ds.hur.sel(plev=70000)
        elif var == 'sfcWind':
            if modelname == 'CCSM4': #calculate ccsm4 WS with ua/va 
                pass
            else:
                ds = ds.sfcWind
        elif var == 'ua':
            if modelname == 'CCSM4':
                ds = ds.ua.sel(plev=1e5)
            else: 
                ds = ds.uas
        elif var == 'va':
            if modelname == 'CCSM4':
                ds = ds.va.sel(plev=1e5)
            else: 
                ds = ds.vas
        elif var == 'wap': 
            ds = ds.wap.sel(plev=70000)
        

        # Change lon coords
        ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
        ds.lat.attrs['units'] = 'degrees' 
        ds.lon.attrs['units'] = 'degrees' 
      
       # Unify time coords to start in year 1 and end in 150
        ds = ds.isel(time=slice(0,150*12))
        
        # Unify time coordinates as datetime64
        if len(filepath)==0: 
            pass 
        elif len(filepath)!=0 :
            if var == 'wap' and modelname == 'HadGEM2':
                index=time_ds.indexes['time'][0:141*12]
            else:
                index=time_ds.indexes['time'][0:150*12]
            ds['time']=index
            
        print(modelname, np.shape(ds))
        
        #interp grids (selected CCSM4 for coordinates)
        kernel_ds = kernel.dRdxi.isel(i=0).expand_dims({'time':time_ds.indexes['time'][0:150*12]}, axis=2)\
            .assign_coords({'latitude':kernel.lat,'longitude':kernel.lon})\
            .rename({'latitude':'lat','longitude':'lon'})
            
        # Interp variables and assign to dict 
        if var == 'ta':
            # interp ts
            ds = ds.interp_like(kernel_ds)
            dict_ds_t700[modelname] = ds
        elif var == 'ts':
            ds = ds.interp_like(kernel_ds)
            dict_ds_ts[modelname] = ds
        elif var == 'hur':
            ds = ds.interp_like(kernel_ds)
            dict_ds_hur700[modelname] = ds
        elif var == 'sfcWind':
            ds = ds.interp_like(kernel_ds)
            dict_ds_sfcWind[modelname] = ds
        elif var == 'ua':
            ds = ds.interp_like(kernel_ds)
            dict_ds_uas[modelname] = ds
        elif var == 'va':
            ds = ds.interp_like(kernel_ds)
            dict_ds_vas[modelname] = ds
        elif var == 'wap':
            ds = ds.interp_like(kernel_ds)
            dict_ds_wap[modelname] = ds
       

CCSM4 3 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ta_Amon_CCSM4_abrupt4xCO2_r1i1p1_185001-189912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ta_Amon_CCSM4_abrupt4xCO2_r1i1p1_190001-194912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ta_Amon_CCSM4_abrupt4xCO2_r1i1p1_195001-200012.nc'] 

CCSM4 (1800, 192, 288)
CCSM4 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ts_Amon_CCSM4_abrupt4xCO2_r1i1p1_185001-200012.nc'] 

CCSM4 (1800, 192, 288)
CCSM4 3 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_CCSM4_abrupt4xCO2_r1i1p1_185001-189912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_CCSM4_abrupt4xCO2_r1i1p1_190001-194912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_CCSM4_abrupt4xCO2_r1i1p1_195001-200012.nc'] 

CCSM4 (1800, 192, 288)
CCSM4 0 
 [] 

CCSM4 (1800, 36, 72)
CCSM4 3 
 ['/data/keeling/a/rytam2/ccf_model_spread/dat

HadGEM2 (1800, 144, 192)
HadGEM2 8 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/vas_Amon_HadGEM2-ES_abrupt4xCO2_r1i1p1_185912-186011.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/vas_Amon_HadGEM2-ES_abrupt4xCO2_r1i1p1_186012-188511.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/vas_Amon_HadGEM2-ES_abrupt4xCO2_r1i1p1_188512-191011.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/vas_Amon_HadGEM2-ES_abrupt4xCO2_r1i1p1_191012-193511.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/vas_Amon_HadGEM2-ES_abrupt4xCO2_r1i1p1_193512-196011.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/vas_Amon_HadGEM2-ES_abrupt4xCO2_r1i1p1_196012-198511.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/vas_Amon_HadGEM2-ES_abrupt4xCO2_r1i1p1_198512-200011.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/vas_Amon_HadGEM2-ES_abrupt4xCO2_r1i1p1_200012-201012.nc'] 



MIROC5 (1800, 128, 256)
MIROC5 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/uas_Amon_MIROC5_abrupt4xCO2_r1i1p1_210001-225012.nc'] 

MIROC5 (1800, 128, 256)
MIROC5 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/vas_Amon_MIROC5_abrupt4xCO2_r1i1p1_210001-225012.nc'] 

MIROC5 (1800, 128, 256)
MPI-ESM 15 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ta_Amon_MPI-ESM-LR_abrupt4xCO2_r1i1p1_185001-185912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ta_Amon_MPI-ESM-LR_abrupt4xCO2_r1i1p1_186001-186912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ta_Amon_MPI-ESM-LR_abrupt4xCO2_r1i1p1_187001-187912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ta_Amon_MPI-ESM-LR_abrupt4xCO2_r1i1p1_188001-188912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ta_Amon_MPI-ESM-LR_abrupt4xCO2_r1i1p1_189001-189912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xC

MRI-CGCM3 (1800, 160, 320)
MRI-CGCM3 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/ts_Amon_MRI-CGCM3_abrupt4xCO2_r1i1p1_185101-200012.nc'] 

MRI-CGCM3 (1800, 160, 320)
MRI-CGCM3 15 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_MRI-CGCM3_abrupt4xCO2_r1i1p1_185101-186012.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_MRI-CGCM3_abrupt4xCO2_r1i1p1_186101-187012.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_MRI-CGCM3_abrupt4xCO2_r1i1p1_187101-188012.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_MRI-CGCM3_abrupt4xCO2_r1i1p1_188101-189012.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_MRI-CGCM3_abrupt4xCO2_r1i1p1_189101-190012.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_MRI-CGCM3_abrupt4xCO2_r1i1p1_190101-191012.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/abrupt4xCO2/hur_Amon_MRI-CGCM3_

In [35]:
import metpy.calc as mpcalc
for modelname in modelfilename_list:
    print(modelname)
    ds = mpcalc.advection(dict_ds_ts[modelname],u=dict_ds_uas[modelname], v=dict_ds_vas[modelname]) 
    dict_ds_tadv[modelname] = ds

CCSM4
CanESM2
CanESM5
E3SM-1-0
GFDL-CM4
HadGEM2
HadGEM3-GC31-LL
IPSL-CM6A-LR
MIROC-ES2L
MIROC-ESM
MIROC5
MIROC6
MPI-ESM
MRI-CGCM3
MRI-ESM2-0
UKESM1-0-LL


In [12]:
# Change dictionaries to Xarray
ts = xr.Dataset(dict_ds_ts)
hur = xr.Dataset(dict_ds_hur700)
ws = xr.Dataset(dict_ds_sfcWind)
wap = xr.Dataset(dict_ds_wap)
tadv = xr.Dataset(dict_ds_tadv)
t700 = xr.Dataset(dict_ds_t700)
uas = xr.Dataset(dict_ds_uas)
vas = xr.Dataset(dict_ds_vas)

In [13]:
# Calcualte EIS 
eis = climlab.EIS(ts,t700)

# Calculate ws from ua/va for CCSM4 
#Metpy has Pint quantity units; chunk removes that 
ws['CCSM4'] = mpcalc.wind_speed(uas.CCSM4,vas.CCSM4).rename('CCSM4').chunk()
ws=ws.drop(['plev','height'])

# Change tadv to dask 
tadv=tadv.chunk()

In [24]:
# Slice time period and save as nc files 
path='/data/keeling/a/rytam2/ccf_model_spread/data/preprocessed/'

ts.to_netcdf(path+'ts_4xCO2_CMIP5&6_Y1-150.nc')
eis.to_netcdf(path+'eis_4xCO2_CMIP5&6_Y1-150.nc')
hur.drop('plev').to_netcdf(path+'hur_4xCO2_CMIP5&6_Y1-150.nc')
ws.to_netcdf(path+'ws_4xCO2_CMIP5&6_Y1-150.nc')
wap.drop('plev').to_netcdf(path+'wap_4xCO2_CMIP5&6_Y1-150.nc')
tadv.to_netcdf(path+'tadv_4xCO2_CMIP5&6_Y1-150.nc')